In [28]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import holidays
# ----------------------------
# 1.  Load and parse the data
# ----------------------------
file_path = "data_loads.csv"          
df = pd.read_csv(file_path)
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

In [29]:
# --- one-hot for weekend and US holidays 
df['is_saturday'] = (df['TIMESTAMP'].dt.weekday == 5).astype(int)
df['is_sunday']   = (df['TIMESTAMP'].dt.weekday == 6).astype(int)

us_holidays = holidays.UnitedStates()
df['is_us_holiday'] = df['TIMESTAMP'].dt.date.apply(lambda d: int(d in us_holidays))

# --- month dummies 
m = df['TIMESTAMP'].dt.month
d = df['TIMESTAMP'].dt.day

month_dummies = pd.get_dummies(m, dtype=int)          # 1 … 12
month_dummies.columns = [f"month_{col:02d}" for col in month_dummies.columns]
df = pd.concat([df, month_dummies], axis=1)




# --- season codes 
is_spring = ((m ==  3) & (d >= 20)) | (m.isin([4, 5])) | ((m == 6) & (d <= 20))
is_summer = ((m ==  6) & (d >= 21)) | (m.isin([7, 8])) | ((m == 9) & (d <= 21))
is_autumn = ((m ==  9) & (d >= 22)) | (m.isin([10, 11])) | ((m == 12) & (d <= 20))

df['season'] = np.select(
    [is_spring, is_summer, is_autumn],
    ['SP',      'SU',      'A'],
    default='W'
)

# --- one-hot seasons 
season_dummies = pd.get_dummies(df['season'], prefix='season', dtype=int)  

df = pd.concat([df, season_dummies], axis=1)
df.drop(columns='season', inplace=True)       




# --- hour dummies 
for hour in range(24):
    df[f'hour_{hour}'] = (df['TIMESTAMP'].dt.hour == hour).astype(int)

print(df)


       ZONEID           TIMESTAMP   LOAD  w1  w2  w3  w4  w5  w6  w7  ...  \
0           1 2005-01-01 01:00:00  125.8  43  46  40  47  48  46  44  ...   
1           1 2005-01-01 02:00:00  121.8  41  46  38  46  48  45  51  ...   
2           1 2005-01-01 03:00:00  117.0  40  46  37  45  45  45  49  ...   
3           1 2005-01-01 04:00:00  114.4  39  46  37  47  48  48  45  ...   
4           1 2005-01-01 05:00:00  113.6  38  46  37  44  48  49  43  ...   
...       ...                 ...    ...  ..  ..  ..  ..  ..  ..  ..  ...   
60595       1 2011-12-09 19:00:00  173.2  55  37  49  52  48  44  45  ...   
60596       1 2011-12-09 20:00:00  175.8  51  37  49  51  46  42  44  ...   
60597       1 2011-12-09 21:00:00  175.6  51  39  51  48  45  43  45  ...   
60598       1 2011-12-09 22:00:00  173.3  52  43  42  46  44  43  44  ...   
60599       1 2011-12-09 23:00:00  167.3  49  45  35  48  43  42  45  ...   

       hour_14  hour_15  hour_16  hour_17  hour_18  hour_19  hour_20  hour_

In [32]:
# 6 --- save back to disk 
df.to_csv("data_loads_final.csv", index=False)

In [21]:
# check that the day 2006-04-17 1:00:00 has in the column hour_1 a value of 1
print(df[(df['TIMESTAMP'] == '2006-04-17 01:00:00') & (df['hour_1'] == 1)])

       ZONEID           TIMESTAMP  LOAD  w1  w2  w3  w4  w5  w6  w7  ...  \
10872       1 2006-04-17 01:00:00  77.4  67  57  63  73  68  60  61  ...   

       hour_14  hour_15  hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  \
10872        0        0        0        0        0        0        0        0   

       hour_22  hour_23  
10872        0        0  

[1 rows x 69 columns]


In [10]:
import pandas as pd

# ------------------------------------------------------------------
# List of legally-observed U.S. federal (“national”) holidays, 2005-2011
# (shifted to the weekday on which they were officially observed).
# ------------------------------------------------------------------
holiday_dates = [
    # 2005
    "2005-01-17", "2005-02-21", "2005-05-30", "2005-07-04",
    "2005-09-05", "2005-10-10", "2005-11-11", "2005-11-24", "2005-12-26",
    # 2006
    "2006-01-02", "2006-01-16", "2006-02-20", "2006-05-29",
    "2006-07-04", "2006-09-04", "2006-10-09", "2006-11-11",
    "2006-11-23", "2006-12-25",
    # 2007
    "2007-01-01", "2007-01-15", "2007-02-19", "2007-05-28",
    "2007-07-04", "2007-09-03", "2007-10-08", "2007-11-12",
    "2007-11-22", "2007-12-25",
    # 2008
    "2008-01-01", "2008-01-21", "2008-02-18", "2008-05-26",
    "2008-07-04", "2008-09-01", "2008-10-13", "2008-11-11",
    "2008-11-27", "2008-12-25",
    # 2009
    "2009-01-01", "2009-01-19", "2009-02-16", "2009-05-25",
    "2009-07-03", "2009-09-07", "2009-10-12", "2009-11-11",
    "2009-11-26", "2009-12-25",
    # 2010
    "2010-01-01", "2010-01-18", "2010-02-15", "2010-05-31",
    "2010-07-05", "2010-09-06", "2010-10-11", "2010-11-11",
    "2010-11-25", "2010-12-24", "2010-12-31",
    # 2011
    "2011-01-17", "2011-02-21", "2011-05-30", "2011-07-04",
    "2011-09-05", "2011-10-10", "2011-11-11", "2011-11-24", "2011-12-26",
]

# ------------------------------------------------------------------
# Verify that every holiday date has is_us_holiday == 1
# ------------------------------------------------------------------
failed = []

for d_str in holiday_dates:
    holiday_date = pd.to_datetime(d_str).date()
    mask = df['TIMESTAMP'].dt.date == holiday_date       # rows for that calendar day

    if not mask.any():
        failed.append(f"{d_str}  —  no rows found")
    elif not (df.loc[mask, 'is_us_holiday'] == 1).all():
        failed.append(f"{d_str}  —  flag missing or 0")

# ------------------------------------------------------------------
# Report
# ------------------------------------------------------------------
if failed:
    print("Problems found with these dates:")
    for msg in failed:
        print("   ", msg)
else:
    print("All holidays correctly flagged with is_us_holiday == 1")


Problems found with these dates:
    2005-01-17  —  no rows found
    2006-01-16  —  no rows found
    2007-01-15  —  no rows found
    2008-01-21  —  no rows found
    2009-01-19  —  no rows found
    2010-01-18  —  no rows found
    2011-01-17  —  no rows found
    2011-12-26  —  no rows found


In [14]:
#print the row where TIMESTAMP is equal to 2005-01-17
print(df[df['TIMESTAMP'] == '2005-05-17 03:00:00'])

      ZONEID           TIMESTAMP  LOAD  w1  w2  w3  w4  w5  w6  w7  ...  \
2834       1 2005-05-17 03:00:00  67.6  62  57  50  64  66  60  63  ...   

      hour_14  hour_15  hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  \
2834        0        0        0        0        0        0        0        0   

      hour_22  hour_23  
2834        0        0  

[1 rows x 57 columns]


In [7]:
#check if there are null values in the column load
if df['LOAD'].isnull().any():
    print("Warning: There are null values in the 'LOAD' column.")

In [8]:
# check if there are null  or missing values in all the columns
if df.isnull().values.any():
    print("Warning: There are null or missing values in the dataframe.")

In [71]:
# print the day 20 of january of 2006
print(df[df['TIMESTAMP'].dt.date == pd.to_datetime('2007-01-20').date()])


       ZONEID           TIMESTAMP   LOAD  w1  w2  w3  w4  w5  w6  w7  ...  \
17759       1 2007-01-20 00:00:00  156.5  43  39  30  37  45  36  42  ...   
17760       1 2007-01-20 01:00:00  152.8  41  39  29  36  43  36  41  ...   
17761       1 2007-01-20 02:00:00  151.2  39  39  28  35  40  34  39  ...   
17762       1 2007-01-20 03:00:00  152.7  37  36  29  36  37  31  38  ...   
17763       1 2007-01-20 04:00:00  156.0  36  32  29  35  36  33  36  ...   
17764       1 2007-01-20 05:00:00  160.6  29  37  29  34  34  30  34  ...   
17765       1 2007-01-20 06:00:00  167.9  36  36  29  33  34  32  33  ...   
17766       1 2007-01-20 07:00:00  178.6  37  34  28  33  34  30  32  ...   
17767       1 2007-01-20 08:00:00  188.8  36  36  27  31  36  32  33  ...   
17768       1 2007-01-20 09:00:00  193.3  39  39  30  38  39  35  38  ...   
17769       1 2007-01-20 10:00:00  187.9  43  45  32  41  43  39  41  ...   
17770       1 2007-01-20 11:00:00  177.7  45  43  35  43  45  42  43  ...   

In [ ]:



# Convert the datetime column to datetime type
# Replace 'date_column_name' with the actual column name in your data
#df['datetime'] = pd.to_datetime(df['datetime'])

# Create US holiday calendar
us_holidays = holidays.UnitedStates()

# Add Saturday and Sunday columns

# Add US holiday column
df['is_us_holiday'] = df['TIMESTAMP'].dt.date.apply(lambda x: int(x in us_holidays))

# View the updated DataFrame
print(df.head())

# Optionally save to new CSV
# df.to_csv("updated_data_loads.csv", index=False)


   ZONEID           TIMESTAMP   LOAD  w1  w2  w3  w4  w5  w6  w7  ...  w18  \
0       1 2005-01-01 01:00:00  125.8  43  46  40  47  48  46  44  ...   50   
1       1 2005-01-01 02:00:00  121.8  41  46  38  46  48  45  51  ...   49   
2       1 2005-01-01 03:00:00  117.0  40  46  37  45  45  45  49  ...   50   
3       1 2005-01-01 04:00:00  114.4  39  46  37  47  48  48  45  ...   49   
4       1 2005-01-01 05:00:00  113.6  38  46  37  44  48  49  43  ...   49   

   w19  w20  w21  w22  w23  w24  w25    Task  is_us_holiday  
0   45   50   47   41   47   47   45  Task 1              1  
1   46   47   47   45   47   47   45  Task 1              1  
2   45   45   46   45   46   46   43  Task 1              1  
3   43   45   44   45   47   47   44  Task 1              1  
4   43   43   45   45   47   46   44  Task 1              1  

[5 rows x 30 columns]


In [28]:
#print where the date is the 24 of november of 2005
print(df[df['TIMESTAMP'].dt.date == pd.to_datetime('2005-11-24').date()])

      ZONEID           TIMESTAMP   LOAD  w1  w2  w3  w4  w5  w6  w7  ...  w18  \
7631       1 2005-11-24 00:00:00  158.3  46  46  51  46  54  47  49  ...   56   
7632       1 2005-11-24 01:00:00  144.9  46  46  51  46  54  48  50  ...   57   
7633       1 2005-11-24 02:00:00  137.2  46  46  51  47  54  48  49  ...   58   
7634       1 2005-11-24 03:00:00  133.5  46  46  50  47  52  47  49  ...   57   
7635       1 2005-11-24 04:00:00  132.5  46  48  48  47  52  48  50  ...   56   
7636       1 2005-11-24 05:00:00  133.6  46  48  45  47  50  48  49  ...   54   
7637       1 2005-11-24 06:00:00  139.3  45  46  46  46  52  47  47  ...   53   
7638       1 2005-11-24 07:00:00  146.6  45  46  46  46  52  47  47  ...   55   
7639       1 2005-11-24 08:00:00  160.5  46  50  45  47  54  49  50  ...   57   
7640       1 2005-11-24 09:00:00  168.4  52  52  52  51  57  53  53  ...   59   
7641       1 2005-11-24 10:00:00  166.7  55  57  55  54  61  55  58  ...   60   
7642       1 2005-11-24 11:0